In [1]:
import numpy as np
import pandas as pd
from random import randint
from copy import deepcopy

In [2]:
# Read in data
with open('../pa3train.txt') as f:
    train = f.readlines()
train = [x.strip() for x in train] 

with open('../pa3test.txt') as f:
    test = f.readlines()
test = [x.strip() for x in test] 

In [3]:
# create empty lists for data sorting
trainData = []
trainLabel = []

testData = []
testLabel = []

In [4]:
# Remove spaces and split into label/data
for i in range (0,len(train)):
    train[i] = train[i].split(" ")
    trainLabel.append(train[i][819])
    trainData.append(train[i][:819])

for i in range (0,len(test)):
    test[i] = test[i].split(" ")
    testLabel.append(test[i][819])
    testData.append(test[i][:819])

In [5]:
# convert label string to int
trainLabel = [int(i) for i in trainLabel]

testLabel = [int(i) for i in testLabel]

In [6]:
# convert data string to int
for i in range(0,len(trainData)):
    trainData[i] = [int(j) for j in trainData[i]]
    
for i in range(0,len(testData)):
    testData[i] = [int(j) for j in testData[i]]

In [7]:
def dotProd(vecA, vecB):
    output = 0
    for i in range (0,len(vecA)):
        mult = vecA[i]*vecB[i]
        output = output + mult
    return output

In [9]:
def regPerceptron(data,label,initW):
    w = initW
    
    # each training data value
    for i in range (1,len(data)):
        location = label[i]*dotProd(w,data[i])
    
        # check if the sign is zero
        if location == 0:
            if(randint(0,1) == 0):
                location = -1
            else:
                location = 1
    
        # if location is negative update w
        if location <= 0:
            # update each feature of w
            for j in range (0,len(data[0])):
                w[j] = w[j] + (label[i]*data[i][j])
                
    return w

In [10]:
def votedPerceptron(data,label,initW):
    # list of all w
    pairList = []
    wList = {1: initW}
    # count of w survival
    c = {1:1}
    counter = 1
    # index of w
    m = 1
    
    # each training data value
    for i in range (1,len(data)):
        location = label[i]*dotProd(wList[m],data[i])
    
        # check if the sign is zero
        if location == 0:
            if(randint(0,1) == 0):
                location = -1
            else:
                location = 1
    
        # if location is negative update w
        if location < 0:
            tempW = []
            # update each feature of new w
            for j in range (0,len(data[0])):
                tempW.append(wList[m][j] + (label[i]*data[i][j]))
            
            # assign w of m+1 to the new w
            wList[m+1] = tempW
            
            # increment w and set c of m to 1
            #print(m)
            #print(c[m])
            m = m + 1
            c[m] = 1
            
            
        # otherwise increment c of m
        else:
            c[m] = c[m]+1
    print(c)
    print(m)
    # make pairs of w and c
    for j in range (1,m):  
        pairList.append((wList[j],c[j]))
        
    return pairList

In [8]:
def calcPreds(data,w):
    preds = []
    into = 0
    for i in range (0,len(data)):
        # clac dot product of w and the training data
        labelPred = dotProd(w,data[i])
    
        # check if dotProd is zero if yes randomly choose a label
        if np.sign(labelPred) == 0:
            into += 1
            if(randint(0,1) == 0):
                preds.append(-1)
            else:
                preds.append(1)
        # greater than 0 means its a positve label
        elif np.sign(labelPred) > 0:
            preds.append(1)
        # less than means negative
        else:
            preds.append(-1)  
    return preds

In [15]:
def calcVotePreds(data,w):
    preds = []
    
    # check all data points
    for i in range (0,len(data)):        
        labelPred = 0
        out = 0
        
        for j in range (0,len(w)):
            # clac dot product of w and the training data
            out = dotProd(w[j][0],data[i])
    
            # check if dotProd is zero if yes randomly choose a sign
            if np.sign(out) == 0:
                if(randint(0,1) == 0):
                    out = -1
                else:
                    out = 1
            # multiply c of j and dotprod result and sum to prediction label
            labelPred = labelPred + (w[j][1]*np.sign(out))
            
        # check if label's sign is zero if yes randomly choose a label
        if np.sign(labelPred) == 0:
            if(randint(0,1) == 0):
                preds.append(-1)
            else:
                preds.append(1)
        # greater than 0 means its a positve label
        elif np.sign(labelPred) > 0:
            preds.append(1)
        # less than means negative
        else:
            preds.append(-1)  
    return preds

In [49]:
def calcAvgPreds(data,w):
    preds = []
    
    cwSum = [0]*len(w[0][0])
    # for all w values in (w,c) pairs
    for j in range (0,len(w)):
        # multiply c and w
        cw = [x * w[j][1] for x in w[j][0]]
        cwSum = [sum(x) for x in zip(cwSum, cw)]  
    
    # check all data points
    for i in range (0,len(data)):        
        labelPred = dotProd(cwSum,data[i])
            
        # check if label's sign is zero if yes randomly choose a label
        if np.sign(labelPred) == 0:
            if(randint(0,1) == 0):
                preds.append(-1)
            else:
                preds.append(1)
        # greater than 0 means its a positve label
        elif np.sign(labelPred) > 0:
            preds.append(1)
        # less than means negative
        else:
            preds.append(-1)  
    return preds

In [32]:
def calcErr(prediction,label):
    wrong = 0
    # calculate # of wrong predictions
    for i in range (0,len(prediction)):
        if prediction[i] != label[i]:
            wrong += 1
    err = wrong/len(prediction)
    return err

In [11]:
# create empty lists for data subset
subTrainData = []
subTrainLabel = []

subTestData = []
subTestLabel = []

subTrainSign = []
subTestSign = []

In [12]:
# grab subsets with label = 1 or 2
for i in range (0,len(trainData)):
    if trainLabel[i] == 1 or trainLabel[i] == 2:
        subTrainData.append(trainData[i])
        subTrainLabel.append(trainLabel[i])
        # map labels to 1 = -1 and 2 = 1
        if trainLabel[i] == 1:
            subTrainSign.append(-1)
        else:
            subTrainSign.append(1)

for i in range (0, len(testData)):
    if testLabel[i] == 1 or testLabel[i] == 2:
        subTestData.append(testData[i])
        subTestLabel.append(testLabel[i])
        # map labels to 1 = -1 and 2 = 1
        if testLabel[i] == 1:
            subTestSign.append(-1)
        else:
            subTestSign.append(1)

In [23]:
# init w to 0
wReg = subTrainData[0]
counts = 0

In [24]:
# each training data value
for i in range (1,len(subTrainData)):
    location = subTrainSign[i]*dotProd(wReg,subTrainData[i])
    
    # check if the sign is zero
    if location == 0:
        if(randint(0,1) == 0):
            location = -1
        else:
            location = 1
    
    # if location is negative update w
    if location < 0:
        # update each feature of w
        counts += 1
        for j in range (0,len(subTrainData[0])):
            wReg[j] = wReg[j] + (subTrainSign[i]*subTrainData[i][j])

In [24]:
newW = regPerceptron(subTrainData,subTrainSign,subTrainData[0])

In [23]:
newW = regPerceptron(subTrainData,subTrainSign,newW)

NameError: name 'newW' is not defined

In [112]:
preds = []
into = 0
for i in range (0,len(subTrainData)):
    # clac dot product of w and the training data
    labelPred = dotProd(newW,subTrainData[i])
    
    # check if dotProd is zero if yes randomly choose a label
    if np.sign(labelPred) == 0:
        into += 1
        if(randint(0,1) == 0):
            preds.append(-1)
        else:
            preds.append(1)
    # greater than 0 means its a positve label
    elif np.sign(labelPred) > 0:
        preds.append(1)
    # less than means negative
    else:
        preds.append(-1)    

In [25]:
out = calcPreds(subTrainData,newW)

In [13]:
vW = votedPerceptron(subTrainData,subTrainSign,subTrainData[0])

{1: 3, 2: 1, 3: 2, 4: 5, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 2, 12: 3, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 2, 21: 1, 22: 5, 23: 2, 24: 2, 25: 20, 26: 12, 27: 1, 28: 1, 29: 32, 30: 11, 31: 3, 32: 1, 33: 4, 34: 3, 35: 1, 36: 6, 37: 57, 38: 3, 39: 2, 40: 2, 41: 1, 42: 4, 43: 1, 44: 2, 45: 9, 46: 1, 47: 2, 48: 1, 49: 2, 50: 3, 51: 14, 52: 3, 53: 7, 54: 1, 55: 6, 56: 12, 57: 2, 58: 13, 59: 1, 60: 2, 61: 6, 62: 1, 63: 12, 64: 15, 65: 1, 66: 1, 67: 2, 68: 41, 69: 4, 70: 8, 71: 4, 72: 13, 73: 4, 74: 4, 75: 1, 76: 7, 77: 2, 78: 1, 79: 4, 80: 3, 81: 2, 82: 1, 83: 1, 84: 2, 85: 1, 86: 1, 87: 7, 88: 1, 89: 1, 90: 1, 91: 2, 92: 1, 93: 2, 94: 1, 95: 1, 96: 2, 97: 3, 98: 1, 99: 32, 100: 9, 101: 7, 102: 3, 103: 17, 104: 24, 105: 4, 106: 7, 107: 4, 108: 3, 109: 4, 110: 10, 111: 5, 112: 18, 113: 6, 114: 13, 115: 5, 116: 6, 117: 3, 118: 7, 119: 1, 120: 42, 121: 14, 122: 18, 123: 9, 124: 5, 125: 7, 126: 12, 127: 11, 128: 2, 129: 30, 130: 9, 131: 4, 132: 29, 133: 3, 134: 8, 135: 2, 13

In [38]:
l = [x * vW[0][1] for x in vW[0][0]]

In [39]:
l

[-3,
 15,
 -6,
 0,
 -6,
 3,
 6,
 9,
 -9,
 15,
 3,
 6,
 3,
 -3,
 -18,
 -3,
 21,
 0,
 -3,
 12,
 15,
 -39,
 9,
 -6,
 9,
 24,
 0,
 -3,
 -9,
 -18,
 12,
 0,
 -18,
 -3,
 27,
 30,
 -3,
 -3,
 -3,
 0,
 3,
 -12,
 -15,
 0,
 -18,
 36,
 -3,
 15,
 -18,
 -24,
 6,
 9,
 -18,
 -18,
 -18,
 3,
 3,
 0,
 69,
 -6,
 3,
 0,
 42,
 15,
 18,
 -24,
 9,
 -12,
 -57,
 0,
 -9,
 9,
 -75,
 -3,
 12,
 51,
 30,
 -3,
 123,
 24,
 0,
 0,
 33,
 9,
 3,
 -18,
 6,
 0,
 3,
 -21,
 0,
 0,
 15,
 0,
 -15,
 0,
 -54,
 6,
 0,
 6,
 -9,
 12,
 -24,
 0,
 -18,
 0,
 9,
 -15,
 30,
 21,
 -24,
 -6,
 3,
 -3,
 -6,
 -6,
 3,
 3,
 -12,
 0,
 -21,
 0,
 -3,
 0,
 12,
 0,
 -6,
 6,
 -6,
 0,
 -3,
 12,
 3,
 -18,
 0,
 -6,
 12,
 3,
 -27,
 12,
 -3,
 27,
 9,
 30,
 0,
 24,
 -3,
 6,
 6,
 0,
 18,
 12,
 3,
 24,
 9,
 24,
 0,
 -3,
 -3,
 0,
 0,
 0,
 0,
 9,
 3,
 -18,
 -3,
 24,
 -45,
 -36,
 -18,
 -9,
 -6,
 9,
 -15,
 0,
 0,
 -9,
 30,
 -15,
 -12,
 -6,
 3,
 -12,
 -3,
 -3,
 12,
 0,
 21,
 -21,
 -6,
 -6,
 -54,
 6,
 9,
 9,
 -6,
 -24,
 -9,
 6,
 9,
 0,
 0,
 -6,
 24,
 0,
 6,
 0,
 21

In [16]:
pred2 = calcVotePreds(subTrainData,vW)

In [51]:
pred3 = calcAvgPreds(subTrainData,vW)

In [52]:
wrong = 0
# calculate # of wrong predictions
for i in range (0,len(pred3)):
    if pred3[i] != subTrainSign[i]:
        wrong += 1

In [55]:
calcErr(pred3,subTrainSign)

0.06788990825688074

In [31]:
err

0.06330275229357799

In [ ]:
dictTry = {1:deepcopy(subTrainData[0])}

In [ ]:
dictTry[1]

In [40]:
counts = {1:0, 2:2}

In [41]:
counts[1] += 1

In [42]:
counts[1]

1

In [43]:
counts[2] = counts[2] +1

In [44]:
counts[2]

3